In [1]:
import sys
print(sys.executable)


/Users/giulia/Documents/BDS/Reinforcement learning/MAB assignments/.venv/bin/python


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np  # Numerical computing primitives for vectorized linear algebra.
import pandas as pd  # Tabular data loading and preprocessing utilities.
from sklearn.feature_extraction import FeatureHasher  # Feature hashing for high-cardinality categorical context.

### Data Loading and Cleaning  
The ZOZOTOWN contextual bandit dataset is loaded from a CSV file and copied to preserve the original data. Column names are standardized to ensure consistent access across the pipeline. The presence of required variables is validated, and key columns are cast to appropriate numeric types to ensure well-defined computations.

### Context Feature Processing  
Contextual information is extracted from user-related variables with prefix `user_feature_`. These variables represent anonymized, high-cardinality categorical attributes and therefore cannot be directly interpreted as numerical features.

### Feature Hashing  
Feature hashing is applied to transform categorical context features into a fixed-dimensional numeric representation. Each context vector is mapped into $\mathbb{R}^d$ with $d = 64$ using a signed hashing scheme to mitigate collision bias. The resulting matrix $X \in \mathbb{R}^{N \times d}$ serves as the numerical context input for the contextual bandit algorithm.


In [ ]:
# ------------------------------------------------------------
# Load + minimal cleanup
# ------------------------------------------------------------
dfZozo_80 = pd.read_csv("zozo_Context_80items.csv")  # Load the ZOZO contextual dataset containing 80 items.

df = dfZozo_80.copy()  # Create a defensive copy to avoid mutating the original DataFrame.
df.columns = (  # Standardize column names for consistent downstream access.
    df.columns.astype(str)  # Ensure the column index is string-typed.
    .str.strip()  # Remove leading and trailing whitespace.
    .str.lower()  # Convert to lowercase for case-insensitive referencing.
    .str.replace(r"\s+", "_", regex=True)  # Replace runs of whitespace with underscores.
    .str.replace(r"[^a-z0-9_]", "", regex=True)  # Remove non-alphanumeric/underscore characters.
)

required = ["item_id", "position", "click", "propensity_score"]  # Define the minimal set of required columns.
missing = [c for c in required if c not in df.columns]  # Identify missing required columns after normalization.
if missing:  # Validate schema conformance before continuing.
    raise ValueError(  # Raise a clear error describing the mismatch.
        f"Missing required columns after cleanup: {missing}. "
        f"Available columns: {list(df.columns)}"
    )

ctx_cols = [c for c in df.columns if c.startswith("user_feature_")]  # Select contextual user feature columns.
if not ctx_cols:  # Ensure context features exist in the dataset.
    raise ValueError("No context columns found. Expected columns like user_feature_0..3")  # Fail fast on schema mismatch.

df["click"] = df["click"].astype(int)  # Cast reward to integer-coded binary labels.
df["propensity_score"] = df["propensity_score"].astype(float)  # Cast logged propensities to floating point.
df["item_id"] = df["item_id"].astype(int)  # Cast item identifiers to integer.
df["position"] = df["position"].astype(int)  # Cast position identifiers to integer.

# ------------------------------------------------------------
# Context preprocessing via feature hashing
# ------------------------------------------------------------
hashed_cols = ctx_cols  # Treat user_feature_* columns as hashed categorical features.
hashed_records = (  # Convert each row into a mapping suitable for FeatureHasher.
    df[hashed_cols]
    .astype(str)  # Ensure all hashed values are represented as strings.
    .to_dict(orient="records")  # Convert to a list of row-wise dictionaries.
)

hash_dim = 64  # Set the target dimensionality for the hashed feature space.
hasher = FeatureHasher(  # Instantiate a feature hasher for scalable categorical encoding.
    n_features=hash_dim,  # Fix the output dimension of the feature hashing transform.
    input_type="dict",  # Indicate that each sample is represented as a dictionary.
    alternate_sign=True  # Use signed hashing to reduce collisions bias.
)

X = hasher.transform(hashed_records).toarray().astype(float)  # Produce a dense numeric context matrix for learning.

Hashed context matrix shape: (1356670, 64)
Sample hashed context row (first 5 features): [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


In [ ]:
dfZozo_80.head()

In [11]:
print(f"Hashed context matrix shape: {X.shape}")  # Log the shape of the resulting context matrix.
print(f"Sample hashed context row (first 5 features): {X[0, :20]}")  # Display a sample hashed context vector.

Hashed context matrix shape: (1356670, 64)
Sample hashed context row (first 5 features): [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Utility Functions  

The logistic sigmoid function maps a real-valued score to a probability in the interval $(0,1)$.  
For an input $z \in \mathbb{R}$, the sigmoid function is defined as
$$
\sigma(z) = \frac{1}{1 + \exp(-z)} .
$$
For numerical stability, the input $z$ is clipped to a bounded range before applying the exponential function.

An action indexing function maps each item–position pair to a unique scalar action identifier.  
Given an item index $i \in \{0,\dots,79\}$ and a position index $p \in \{0,1,2\}$, the action ID is defined as
$$
a = i \times n_{\text{positions}} + p ,
$$
where $n_{\text{positions}} = 3$.

As an illustrative example:
- for item $i = 0$ and positions $p = 0,1,2$,
$$
a = 0 \times 3 + p \in \{0,1,2\},
$$
- for item $i = 1$,
$$
a = 1 \times 3 + p \in \{3,4,5\},
$$
- for item $i = 79$,
$$
a = 79 \times 3 + p \in \{237,238,239\}.
$$

Therefore, the action index satisfies
$$
a \in \{0,\dots,239\},
$$
and the total number of actions is $80 \times 3 = 240$.


In [12]:
# ------------------------------------------------------------
# Utilities
# ------------------------------------------------------------
def sigmoid(z: np.ndarray) -> np.ndarray:  # Define the logistic sigmoid function.
    z = np.clip(z, -35, 35)  # Clip logits to prevent numerical overflow in exp().
    return 1.0 / (1.0 + np.exp(-z))  # Compute elementwise sigmoid.

def make_action_id(item_id: np.ndarray, position: np.ndarray, n_positions: int = 3) -> np.ndarray:  # Map (item,position) to a single action index.
    item0 = item_id.copy()  # Copy to avoid side effects on the input array.
    if item0.min() == 1:  # Normalize 1-based item indexing to 0-based indexing if required.
        item0 = item0 - 1  # Convert item IDs from {1..80} to {0..79}.
    pos0 = position - 1  # Convert position IDs from {1,2,3} to {0,1,2}.
    return item0 * n_positions + pos0  # Compute unique action IDs in [0, 80*3-1].

### Logistic Thompson Sampling (LogTS)

Logistic Thompson Sampling is implemented with an action-specific diagonal Gaussian posterior over the logistic regression parameters.  
For each action $a \in \{0,\dots,K-1\}$, the posterior approximation is
$$
\theta_a \sim \mathcal{N}\!\left(m_a,\;\mathrm{diag}(q_a)^{-1}\right),
$$
where $m_a \in \mathbb{R}^d$ is the posterior mean and $q_a \in \mathbb{R}^d$ is the diagonal precision vector.

An optional intercept term is included by augmenting the context vector $x \in \mathbb{R}^{d-1}$ into
$$
\tilde{x} = \begin{bmatrix} x \\ 1 \end{bmatrix} \in \mathbb{R}^{d}.
$$

At each round, LogTS performs posterior sampling for action selection. A parameter vector is sampled for each action,
$$
\tilde{\theta}_a \sim \mathcal{N}\!\left(m_a,\;\mathrm{diag}(q_a)^{-1}\right),
$$
and the click probability under the sampled parameters is computed as
$$
p_a = \sigma(\tilde{\theta}_a^\top \tilde{x}).
$$
The selected action is
$$
\hat{a} = \arg\max_{a} \; p_a.
$$

Only partial feedback is available in logged bandit data. The posterior update is applied only when the selected action matches the logged action $a_i$.

The posterior mean update is implemented as a one-step Newton approximation to the MAP estimate under a diagonal Gaussian prior centered at the current mean.  
Let $w$ denote the current iterate, and define
$$
p = \sigma(w^\top \tilde{x}).
$$
The gradient of the negative log-posterior is
$$
g = q \odot (w - m_0) + (p - r)\tilde{x},
$$
where $m_0$ is the current mean, $q$ is the current diagonal precision, and $\odot$ denotes elementwise multiplication.  
The Hessian has the form
$$
H = \mathrm{diag}(q) + p(1-p)\tilde{x}\tilde{x}^\top,
$$
and the Newton step updates $w \leftarrow w - H^{-1}g$.

After updating the mean, the diagonal precision is updated using a Laplace-style curvature approximation:
$$
q \leftarrow q + \tilde{x}^{\odot 2}\, p(1-p),
$$
where $\tilde{x}^{\odot 2}$ denotes elementwise squaring.

For off-policy evaluation, the action selection probability $\pi_e(a\mid x)$ is approximated by Monte Carlo posterior sampling.  
For $M$ samples, the policy probability is estimated as
$$
\widehat{\pi}_e(a\mid x) = \frac{1}{M}\sum_{m=1}^{M} \mathbf{1}\!\left[a = \arg\max_{a'} \sigma\!\left(\tilde{\theta}^{(m)}_{a'}{}^\top \tilde{x}\right)\right].
$$


In [13]:
# ------------------------------------------------------------
# LogTS implementation (diagonal Gaussian posterior per action)
# Posterior: theta_a ~ N(m_a, diag(1/q_a))
# Online update (batch size 1) with partial feedback:
# - Sample theta_a for each action.
# - Select action maximizing sigmoid(theta_a^T x).
# - Update only when the selected action matches the logged action.
# - Apply a one-step Newton update toward the MAP estimate under a diagonal Gaussian prior.
# - Update diagonal precision via p(1-p) x^2.
# ------------------------------------------------------------
class LogTS:  # Define a Logistic Thompson Sampling policy with diagonal Gaussian posteriors.
    def __init__(  # Initialize model state and prior parameters.
        self,
        n_actions: int,
        context_dim: int,
        lam: float = 1.0,
        seed: int = 42,
        add_intercept: bool = True,
    ):
        self.n_actions = n_actions  # Store number of discrete actions.
        self.add_intercept = add_intercept  # Indicate whether to append an intercept to context vectors.
        self.rng = np.random.default_rng(seed)  # Initialize a reproducible random number generator.

        self.d = context_dim + (1 if add_intercept else 0)  # Determine parameter dimension, including intercept if enabled.

        self.m = np.zeros((n_actions, self.d), dtype=float)  # Initialize posterior means for all actions.
        self.q = np.full((n_actions, self.d), lam, dtype=float)  # Initialize diagonal precisions for all actions.

    def _augment_x(self, x: np.ndarray) -> np.ndarray:  # Augment a context vector with an intercept term if required.
        if not self.add_intercept:  # Skip augmentation when intercept is disabled.
            return x  # Return the context unchanged.
        return np.concatenate([x, np.array([1.0])], axis=0)  # Append a constant intercept feature.

    def sample_thetas(self) -> np.ndarray:  # Sample parameter vectors for all actions from their posteriors.
        std = 1.0 / np.sqrt(self.q)  # Convert diagonal precision to standard deviation.
        return self.m + self.rng.normal(size=self.m.shape) * std  # Draw Normal samples for all actions.

    def choose_action(self, x: np.ndarray) -> int:  # Select an action using posterior sampling and logistic scoring.
        x_aug = self._augment_x(x)  # Construct the augmented context vector.
        thetas = self.sample_thetas()  # Draw one parameter sample per action.
        logits = thetas @ x_aug  # Compute sampled logits for each action.
        probs = sigmoid(logits)  # Convert logits to click probabilities.
        return int(np.argmax(probs))  # Choose the action with the highest sampled probability.

    def update_selected_action(self, a: int, x: np.ndarray, r: int, newton_steps: int = 1):  # Update the posterior of a selected action using one-step online Bayesian logistic regression.
        x_aug = self._augment_x(x)  # Construct the augmented context vector.

        m0 = self.m[a].copy()  # Store the prior mean (current posterior mean) for action a.
        q = self.q[a]  # Retrieve diagonal precision vector for action a.
        w = m0.copy()  # Initialize the Newton iterate at the prior mean.

        for _ in range(newton_steps):  # Perform a small number of Newton steps for a MAP approximation.
            p = float(sigmoid(np.array([w @ x_aug]))[0])  # Evaluate sigmoid at the current iterate.

            g = q * (w - m0) + (p - r) * x_aug  # Compute gradient of negative log-posterior under diagonal Gaussian prior.

            s = p * (1.0 - p)  # Compute curvature term for the logistic likelihood.
            D_inv = 1.0 / q  # Compute inverse of diagonal precision for Sherman-Morrison update.
            Dx = D_inv * x_aug  # Compute D^{-1} x for the rank-one correction.
            denom = 1.0 + s * float(x_aug @ Dx)  # Compute scalar denominator 1 + s x^T D^{-1} x.

            Dinv_g = D_inv * g  # Compute D^{-1} g.
            corr = (s / denom) * (Dx * float(x_aug @ Dinv_g))  # Compute Sherman-Morrison correction term.
            delta = Dinv_g - corr  # Solve for Newton step delta = (D + s x x^T)^{-1} g.

            w = w - delta  # Apply Newton update to the iterate.

        self.m[a] = w  # Commit updated posterior mean for action a.

        p_new = float(sigmoid(np.array([w @ x_aug]))[0])  # Compute probability at the updated mean.
        s_new = p_new * (1.0 - p_new)  # Compute logistic curvature at the updated mean.
        self.q[a] = q + (x_aug ** 2) * s_new  # Update diagonal precision using a Laplace-style approximation.

    def fit_online(self, X: np.ndarray, A_logged: np.ndarray, R: np.ndarray, n_rounds: int = None):  # Simulate LogTS over a sequence of contexts with partial feedback updates.
        n = X.shape[0] if n_rounds is None else min(n_rounds, X.shape[0])  # Determine the number of rounds to simulate.
        chosen = np.empty(n, dtype=int)  # Allocate output array for chosen actions.

        for i in range(n):  # Iterate over interaction rounds.
            a_hat = self.choose_action(X[i])  # Select an action under LogTS for context i.
            chosen[i] = a_hat  # Store the selected action.

            if a_hat == A_logged[i]:  # Check whether the selected action matches the logged action.
                self.update_selected_action(a_hat, X[i], int(R[i]), newton_steps=1)  # Update posterior using observed reward for the matching action.

        return chosen  # Return the counterfactual action log produced by LogTS.

    def action_probs_mc(self, x: np.ndarray, n_mc: int = 200) -> np.ndarray:  # Approximate pi_e(a|x) by Monte Carlo posterior sampling with argmax selection.
        x_aug = self._augment_x(x)  # Construct the augmented context vector.
        counts = np.zeros(self.n_actions, dtype=float)  # Initialize action selection counts.

        std = 1.0 / np.sqrt(self.q)  # Convert diagonal precision to standard deviation for sampling.
        for _ in range(n_mc):  # Repeat sampling to approximate selection probabilities.
            thetas = self.m + self.rng.normal(size=self.m.shape) * std  # Sample one parameter vector per action.
            a = int(np.argmax(sigmoid(thetas @ x_aug)))  # Select the action with highest sampled click probability.
            counts[a] += 1.0  # Increment selection count for the chosen action.

        return counts / n_mc  # Normalize counts to obtain an empirical action probability distribution.

In [14]:
# ------------------------------------------------------------
# OPE estimators: IPW and SNIPW + bootstrap CI
# ------------------------------------------------------------
def ipw(pi_e: np.ndarray, pi_b: np.ndarray, r: np.ndarray) -> float:  # Compute the inverse probability weighting estimate of the policy value.
    w = pi_e / np.clip(pi_b, 1e-12, None)  # Compute importance weights with numerical protection against division by zero.
    return float(np.mean(w * r))  # Return the sample mean of weighted rewards.

def snipw(pi_e: np.ndarray, pi_b: np.ndarray, r: np.ndarray) -> float:  # Compute the self-normalized IPW estimate of the policy value.
    w = pi_e / np.clip(pi_b, 1e-12, None)  # Compute importance weights with numerical protection against division by zero.
    num = float(np.sum(w * r))  # Compute weighted reward sum.
    den = float(np.sum(w))  # Compute weight sum for normalization.
    return num / den if den > 0 else 0.0  # Return normalized estimate, guarding against a zero denominator.

def bootstrap_ci(values: np.ndarray, alpha: float = 0.05):  # Compute a two-sided percentile bootstrap confidence interval.
    lo = float(np.quantile(values, alpha / 2.0))  # Compute lower percentile bound.
    hi = float(np.quantile(values, 1.0 - alpha / 2.0))  # Compute upper percentile bound.
    return lo, hi  # Return the confidence interval endpoints.

def ope_with_bootstrap(pi_e: np.ndarray, pi_b: np.ndarray, r: np.ndarray, B: int = 100, seed: int = 7):  # Evaluate IPW and SNIPW with bootstrap confidence intervals.
    rng = np.random.default_rng(seed)  # Initialize reproducible bootstrap sampling.
    n = len(r)  # Determine sample size.

    ipw_vals = np.empty(B, dtype=float)  # Allocate bootstrap storage for IPW estimates.
    snipw_vals = np.empty(B, dtype=float)  # Allocate bootstrap storage for SNIPW estimates.

    for b in range(B):  # Perform bootstrap resampling.
        idx = rng.integers(0, n, size=n)  # Draw bootstrap indices with replacement.
        ipw_vals[b] = ipw(pi_e[idx], pi_b[idx], r[idx])  # Compute IPW estimate on bootstrap sample.
        snipw_vals[b] = snipw(pi_e[idx], pi_b[idx], r[idx])  # Compute SNIPW estimate on bootstrap sample.

    return {  # Return point estimates and percentile confidence intervals.
        "ipw": float(ipw(pi_e, pi_b, r)),  # Compute IPW point estimate on full data.
        "ipw_ci95": bootstrap_ci(ipw_vals, alpha=0.05),  # Compute 95% bootstrap CI for IPW.
        "snipw": float(snipw(pi_e, pi_b, r)),  # Compute SNIPW point estimate on full data.
        "snipw_ci95": bootstrap_ci(snipw_vals, alpha=0.05),  # Compute 95% bootstrap CI for SNIPW.
    }

### End-to-End Pipeline: LogTS Simulation and Off-Policy Evaluation

The logged action $a_i$ is constructed from the item and position fields by mapping each $(\text{item\_id}, \text{position})$ pair to a unique action index. The logged binary reward is extracted as $r_i \in \{0,1\}$, and the behavior policy propensity $\pi_b(a_i\mid x_i)$ is read from the propensity score column.

The action space size is defined as
$$
K = 80 \times 3 = 240,
$$
corresponding to 80 items and 3 display positions.

A Logistic Thompson Sampling policy is instantiated with context dimension $d$ equal to the number of hashed context features, and with a diagonal Gaussian prior precision controlled by $\lambda$. The policy is then simulated sequentially over the logged contexts $\{x_i\}_{i=1}^{N}$ using partial-feedback updates, i.e., posterior updates are applied only when the selected action matches the logged action.

For off-policy evaluation, the evaluation policy probability of the logged action, $\pi_e(a_i\mid x_i)$, is required for each interaction. Since LogTS defines $\pi_e$ implicitly through posterior sampling and an argmax decision rule, $\pi_e(a_i\mid x_i)$ is approximated via Monte Carlo:
$$
\widehat{\pi}_e(a_i\mid x_i) = \frac{1}{M}\sum_{m=1}^{M} 
\mathbf{1}\!\left[a_i = \arg\max_{a} \sigma\!\left(\tilde{\theta}^{(m)}_{a}{}^\top \tilde{x}_i\right)\right],
$$
where $M$ is the number of Monte Carlo samples, $\tilde{\theta}^{(m)}_a$ is a posterior sample for action $a$, and $\tilde{x}_i$ denotes the context vector augmented with an intercept if applicable.

Given $\widehat{\pi}_e(a_i\mid x_i)$, the policy value is estimated using importance sampling estimators. The IPW estimator is
$$
\widehat{V}_{\mathrm{IPW}}(\pi_e)
= \frac{1}{N}\sum_{i=1}^{N}
\frac{\widehat{\pi}_e(a_i\mid x_i)}{\pi_b(a_i\mid x_i)}\, r_i,
$$
and the SNIPW estimator is
$$
\widehat{V}_{\mathrm{SNIPW}}(\pi_e)
=
\frac{\sum_{i=1}^{N}\frac{\widehat{\pi}_e(a_i\mid x_i)}{\pi_b(a_i\mid x_i)}\, r_i}
{\sum_{i=1}^{N}\frac{\widehat{\pi}_e(a_i\mid x_i)}{\pi_b(a_i\mid x_i)}}.
$$

Uncertainty is quantified using non-parametric bootstrap with $B=100$ resamples, producing percentile-based $95\%$ confidence intervals for both IPW and SNIPW estimates.


In [17]:
# ------------------------------------------------------------
# End-to-end (full data): simulate LogTS, compute OPE with reduced MC/bootstrap
# ------------------------------------------------------------

M_fast = 5     # Monte Carlo samples per context (reduce from 200)
B_fast = 20    # bootstrap resamples (reduce from 100)

# Logged arrays over the full dataset
A_logged = make_action_id(
    df["item_id"].to_numpy(),
    df["position"].to_numpy(),
    n_positions=3
)
R = df["click"].to_numpy().astype(int)
pi_b = df["propensity_score"].to_numpy().astype(float)

K = 80 * 3

# Simulate LogTS over the full dataset (partial-feedback updates)
logts = LogTS(
    n_actions=K,
    context_dim=X.shape[1],
    lam=1.0,
    seed=123,
    add_intercept=True
)

chosen_actions = logts.fit_online(
    X=X,
    A_logged=A_logged,
    R=R,
    n_rounds=None
)

print("LogTS simulation finished.")

# ------------------------------------------------------------
# Approximate pi_e(a_i | x_i) for all i using reduced Monte Carlo
# ------------------------------------------------------------
N = len(df)
pi_e_logged = np.empty(N, dtype=float)

for i in range(N):
    probs = logts.action_probs_mc(X[i], n_mc=M_fast)
    pi_e_logged[i] = probs[A_logged[i]]

print("pi_e(a_i|x_i) estimated for all rows.")

# ------------------------------------------------------------
# OPE with reduced bootstrap
# ------------------------------------------------------------
ope_results = ope_with_bootstrap(
    pi_e=pi_e_logged,
    pi_b=pi_b,
    r=R,
    B=B_fast,
    seed=99
)

print("OPE results (reduced MC/bootstrap):")
print(ope_results)


LogTS simulation finished.
pi_e(a_i|x_i) estimated for all rows.
OPE results (reduced MC/bootstrap):
{'ipw': 0.0005425048095704924, 'ipw_ci95': (0.0004009818157694944, 0.000674003257977253), 'snipw': 0.00275498592561538, 'snipw_ci95': (0.002049045604013863, 0.0034229662089410758)}


In [ ]:
# ------------------------------------------------------------
# Small-case sanity check (fast)
# ------------------------------------------------------------
A_logged = make_action_id(  # Construct logged action identifiers from item and position columns.
    df["item_id"].to_numpy(),  # Extract item IDs as a NumPy array.
    df["position"].to_numpy(),  # Extract positions as a NumPy array.
    n_positions=3  # Specify the number of available positions.
)

R = df["click"].to_numpy().astype(int)  # Extract logged rewards as a binary integer array.
pi_b = df["propensity_score"].to_numpy().astype(float)  # Extract behavior policy propensities for logged actions.


N_small = 1000      # number of logged interactions
M_small = 10        # Monte Carlo samples
B_small = 10        # bootstrap resamples

# Slice the data
X_small = X[:N_small]
A_logged_small = A_logged[:N_small]
R_small = R[:N_small]
pi_b_small = pi_b[:N_small]

# Instantiate LogTS
logts_small = LogTS(
    n_actions=K,
    context_dim=X.shape[1],
    lam=1.0,
    seed=123,
    add_intercept=True
)

# Run online simulation
chosen_actions_small = logts_small.fit_online(
    X=X_small,
    A_logged=A_logged_small,
    R=R_small,
    n_rounds=N_small
)

print("Simulation finished on small dataset.")

# ------------------------------------------------------------
# Approximate pi_e(a_i | x_i) with small MC
# ------------------------------------------------------------
pi_e_logged_small = np.empty(N_small, dtype=float)

for i in range(N_small):
    probs = logts_small.action_probs_mc(X_small[i], n_mc=M_small)
    pi_e_logged_small[i] = probs[A_logged_small[i]]

print("Policy probabilities estimated.")

# ------------------------------------------------------------
# OPE with small bootstrap
# ------------------------------------------------------------
ope_results_small = ope_with_bootstrap(
    pi_e=pi_e_logged_small,
    pi_b=pi_b_small,
    r=R_small,
    B=B_small,
    seed=99
)

print("Small-case OPE results:")
print(ope_results_small)


Simulation finished on small dataset.
Policy probabilities estimated.
Small-case OPE results:
{'ipw': 0.008, 'ipw_ci95': (0.0, 0.022200000000000004), 'snipw': 0.027777777777777776, 'snipw_ci95': (0.0, 0.06306818181818183)}


In [15]:
# ------------------------------------------------------------
# End-to-end: prepare arrays, simulate LogTS, compute OPE
# ------------------------------------------------------------
A_logged = make_action_id(  # Construct logged action identifiers from item and position columns.
    df["item_id"].to_numpy(),  # Extract item IDs as a NumPy array.
    df["position"].to_numpy(),  # Extract positions as a NumPy array.
    n_positions=3  # Specify the number of available positions.
)
R = df["click"].to_numpy().astype(int)  # Extract logged rewards as a binary integer array.
pi_b = df["propensity_score"].to_numpy().astype(float)  # Extract behavior policy propensities for logged actions.

K = 80 * 3  # Define the number of actions as item-position pairs.

logts = LogTS(  # Instantiate a LogTS policy with hashed context inputs.
    n_actions=K,  # Set the size of the action space.
    context_dim=X.shape[1],  # Set the dimensionality of the hashed context vector.
    lam=1.0,  # Set the initial diagonal precision (regularization strength).
    seed=123,  # Set random seed for reproducible sampling.
    add_intercept=True  # Enable an intercept feature for the logistic model.
)

chosen_actions = logts.fit_online(  # Simulate LogTS online over the logged contexts.
    X=X,  # Provide the hashed context matrix.
    A_logged=A_logged,  # Provide the logged action IDs.
    R=R,  # Provide the logged rewards.
    n_rounds=None  # Simulate the full dataset length.
)

n_mc = 200  # Specify the number of Monte Carlo samples for approximating pi_e(a|x).
pi_e_logged = np.empty(len(df), dtype=float)  # Allocate array for evaluation policy probability of the logged action.

for i in range(len(df)):  # Iterate over all logged interactions to compute pi_e(a_i|x_i).
    probs = logts.action_probs_mc(X[i], n_mc=n_mc)  # Approximate the evaluation policy distribution over actions for context x_i.
    pi_e_logged[i] = probs[A_logged[i]]  # Extract probability mass assigned to the logged action a_i.

ope_results = ope_with_bootstrap(  # Compute OPE estimates with bootstrap confidence intervals.
    pi_e=pi_e_logged,  # Provide evaluation policy probability for the logged action per row.
    pi_b=pi_b,  # Provide behavior policy propensity per row.
    r=R,  # Provide observed rewards.
    B=100,  # Specify the number of bootstrap resamples.
    seed=99  # Set bootstrap RNG seed for reproducibility.
)

print(ope_results)  # Print the resulting IPW and SNIPW estimates and confidence intervals.

KeyboardInterrupt: 